In [7]:
from source.chroma_manager import ChromaDBManager, CustomEmbeddingFunction
from source.llm_manager import LMStudioClient
from prompt.prompts import QuestionsToDoc
from tqdm import tqdm
import ast
import json
import logging

DB_CONFIG_PATH = "config/embedding/questions_gen.yaml"
MODEL_CONFIG_PATH = "config/model_question_gen.yaml"

logging.basicConfig(filename="log/question_gen.log", level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s")


# Инициализация менеджеров
source_chroma = ChromaDBManager(DB_CONFIG_PATH)

llm = LMStudioClient(MODEL_CONFIG_PATH)  # URL настроить под свой сервер

# Извлечение всех документов
all_documents = source_chroma.collection.get()
doc_ids = all_documents.get("ids", [])
doc_texts = all_documents.get("documents", [])
doc_metadata = all_documents.get("metadatas", [])

In [8]:
len(doc_ids)

436

In [9]:
# doc_ids_batch = doc_ids[300:]
# doc_texts_batch = doc_texts[300:]
# doc_metadata_batch = doc_metadata[300:]

In [10]:
def safe_str_to_dict(s: str):
    """
    Преобразует строку в словарь, если возможно.
    В противном случае возвращает False.
    """
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return result
        return False
    except (ValueError, SyntaxError):
        return False

In [11]:
# Список ids где в метадате отсутвуют вопросы
ids_to_generate = []
for ids, metadatas in zip(doc_ids, doc_metadata):
    if 'questions' not in list(metadatas.keys()):
        ids_to_generate.append(ids)
len(ids_to_generate)

0

In [5]:
for doc_id, doc_text, metadata in tqdm(zip(doc_ids, doc_texts, doc_metadata)):
    if doc_id in ids_to_generate:
        # Генерация вопросов через LLM
        prompt = QuestionsToDoc.QUESTION_FORMATION.format(document=doc_text)
        # print(repr(prompt))  # Выведет строку в raw-формате
        questions = llm.post_completion(user_input=prompt)
        llm.clear_context() # очищаем контекст
        questions_dict = safe_str_to_dict(questions)
        if questions_dict == False:
            prompt = QuestionsToDoc.CORRECTING_DICKTIONARY.format(input=questions)
            questions = llm.post_completion(user_input=prompt)
            llm.clear_context() # очищаем контекст
            questions_dict = safe_str_to_dict(questions)
            if questions_dict == False:
                print("Словарь не сгенерирован")
                print(questions)
                logging.error(f" doc_id={doc_id}: questions_dict={questions}")
                continue
            
        # Обновление метаданных
        # print("Успешно")
        # print(questions_dict)
        metadata["questions"] = json.dumps(questions_dict)
        source_chroma.update_document(document_id=doc_id, new_text=doc_text, new_metadata=metadata)

0it [00:00, ?it/s]

32it [10:13, 25.34s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:4: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
39it [13:16, 28.94s/it]<unknown>:2: SyntaxWarning: invalid escape sequence '\('
<unknown>:3: SyntaxWarning: invalid escape sequence '\('
93it [57:31, 29.63s/it] <unknown>:3: SyntaxWarning: invalid escape sequence '\('
110it [1:05:56, 28.01s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:4: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
179it [1:48:26, 21.26s/it] <unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
220it [2:23:40, 22.79s/it] <unknown>:3: SyntaxWarning: invalid escape sequence '\('
232it [2:28:14, 25.46s/it]<unknown>:5: SyntaxWarning: invalid escape sequence '\('
250it [2:36:32, 32.17s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
